In [1]:
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
##Client Authorization

cid = "26e46f3a7818494286f6349bd492484d"
secret = "f96866fd97af42b38349b865821e631f"
ruri = "http://localhost:64664/callback"

scope = "user-library-read"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret = secret)
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope, client_id = cid, client_secret = secret, redirect_uri = ruri))

In [23]:
##function to add tracks to a dataframe
def track_adder(dataframe, track_dict): ##where dataframe is the master dataframe and track_dict a dictionary with structure {playlist title: [uri, mood]}
    for playlist in track_dict:
        plist = sp.playlist_tracks(track_dict[playlist][0]) ##gets the json entry for the playlist from the api
        i = 0
        while i < len(plist["items"]): ##crawl through the tracks in items
            dataframe = dataframe.append({"track": plist["items"][i]["track"]["name"], "uri": plist["items"][i]["track"]["uri"], "mood": track_dict[playlist][1]}, ignore_index = True) ##adds each track info to the dataframe
            i = i+1
    return dataframe

In [ ]:
##function to add track features to data frame
def feature_adder(dataframe): ##just needs a data frame 
    for index, row in dataframe.iterrows():
        features = sp.audio_features(row[1]) ##pulls the audio features and fills them in accordingly
        row[3] = features[0]["danceability"]
        row[4] = features[0]["energy"]
        row[5] = features[0]["key"]
        row[6] = features[0]["loudness"]
        row[7] = features[0]["mode"]
        row[8] = features[0]["speechiness"]
        row[9] = features[0]["acousticness"]
        row[10] = features[0]["instrumentalness"]
        row[11] = features[0]["liveness"]
        row[12] = features[0]["valence"]
        row[13] = features[0]["tempo"]
    return dataframe

In [ ]:
##function to create our database
def create_database(trackDict): ##track_dict a dictionary with structure {playlist title: [uri, mood]}
    tracks = pd.DataFrame(columns = ["track", "uri", "mood", "danceability", "energy", "key","loudness", "mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"], index = None)
    tracks = track_adder(tracks, trackDict)
    tracks = feature_adder(tracks)
    return tracks

In [33]:
##testing all the functions
## testing track_dict
testing = {"happy": ["spotify:playlist:574pNsFHHsY8Dp12ppLiim", "happy"], "sad": ["spotify:playlist:4po6P93XjiQ3CC3UB9UlWV", "sad"]}
create_database(testing)

,track,uri,mood,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Electric Love,spotify:track:2GiJYvgVaD2HtM8GqD9EgQ,happy,0.611,0.797,6,-7.627,0,0.0533,0.00543,0.00137,0.26,0.518,120.041
1,SUPERBLOOM,spotify:track:2HvtedoEeymVWrBPyAMNwZ,happy,0.558,0.889,8,-3.945,1,0.102,0.191,0.000387,0.328,0.716,170.102
2,Let's Fall in Love for the Night,spotify:track:7kQkmyoHCEqwe7QwDbkSXM,happy,0.737,0.408,5,-7.941,1,0.104,0.802,0,0.171,0.374,127.921
3,The Bones - with Hozier,spotify:track:1yTTMcUhL7rtz08Dsgb7Qb,happy,0.561,0.597,11,-6,0,0.0405,0.286,0,0.0979,0.355,76.826
4,Black And White,spotify:track:7rpNuuoMbid56XkDsx2FjE,happy,0.443,0.582,2,-6.02,1,0.0482,0.0301,0,0.127,0.239,147.589
5,Oil & Water,spotify:track:4HwJ2gvewwzC3NsC9WwmlA,sad,0.577,0.377,10,-6.669,1,0.0325,0.699,0,0.128,0.238,126.368
6,Coping,spotify:track:0RjKWagIjeOoxpsiCT4gls,sad,0.502,0.444,6,-7.294,1,0.116,0.613,0,0.103,0.196,157.604
7,Routine,spotify:track:5oRAL1SuCwcdDx6IwK54kL,sad,0.641,0.249,4,-13.669,1,0.0733,0.748,0,0.316,0.109,85.998
8,For Now,spotify:track:5wV2TAUTKHcKgaEB6XnWDI,sad,0.731,0.233,1,-11.109,1,0.0376,0.89,0,0.3,0.408,108.008
9,Swollen (feat. Julia Gargano),spotify:track:1XZG6gPL5pKw7MLRXCK1yX,sad,0.447,0.376,10,-9.262,1,0.0316,0.837,0.000222,0.116,0.219,140.757
